In [18]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import models
from keras import layers
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import pyswarms as ps
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)

In [10]:
df = pd.read_csv('../Datasets/Final_Dataset.csv', index_col=[0])

In [11]:
df.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,gender,jaundice,PDD,Class/ASD,age_range_'12-16 years',age_range_'18 and more',age_range_'4-11 years'
0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,1,0
1,1,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0
2,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,1,0
3,1,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [12]:
df.shape

(1100, 17)

In [13]:
labels = df.pop('Class/ASD')
y = np.array(labels)

In [14]:
X = df.to_numpy()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

In [16]:
print(f'We have {X_train.shape[0]} train samples')
print(f'We have {X_test.shape[0]} test samples')

We have 880 train samples
We have 220 test samples


In [19]:
network = models.Sequential()
network.add(layers.Dense(units=32, activation='sigmoid', input_shape=(X_train.shape[1],)))
network.add(layers.Dense(units=32, activation='sigmoid'))
network.add(layers.Dense(units=1, activation='sigmoid'))

2020-05-02 18:26:36,987 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2020-05-02 18:26:36,991 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2020-05-02 18:26:36,996 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [20]:
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [21]:
def minimization_function(x):
#     print(len(x))
    layer1_weights = np.array(x[0:512]).reshape(16,32)
    layer1_biases = np.array(x[512:544])
    
    layer2_weights = np.array(x[544:1568]).reshape(32,32)
    layer2_biases = np.array(x[1568:1600])
    
    layer3_weights = np.array(x[1600:1632]).reshape(32,1)
    layer3_biases = np.array(x[1632:1633])
    
    network.layers[0].set_weights([layer1_weights, layer1_biases])
    network.layers[1].set_weights([layer2_weights, layer2_biases])
    network.layers[2].set_weights([layer3_weights, layer3_biases])
    
    cso_predict = network.predict(X_train)    
    return mean_squared_error(y_train, cso_predict)

In [23]:
def swarm_function(x):
    n_particles = x.shape[0]
    j = [minimization_function(x[i]) for i in range(n_particles)]
    return np.array(j)

In [24]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=1633, options=options)
cost, pos = optimizer.optimize(swarm_function, iters=100)

2020-05-02 18:27:46,627 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/100

2020-05-02 18:27:46,630 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



2020-05-02 18:27:46,631 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



2020-05-02 18:27:46,633 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



2020-05-02 18:27:46,637 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



2020-05-02 18:27:46,639 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.



2020-05-02 18:27:46,684 - tensorflow - WARNING - From /Users/rajas/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.0234
2020-05-02 18:29:58,068 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.023426681918342723, best pos: [-0.23435307 -1.22267425 -0.11845802 ...  0.08101619  0.18598424
  1.73955696]


In [25]:
print(cost)

0.023426681918342723


In [26]:
print(pos)

[-0.23435307 -1.22267425 -0.11845802 ...  0.08101619  0.18598424
  1.73955696]


In [28]:
layer1_weights = np.array(pos[0:512]).reshape(16,32)
layer1_biases = np.array(pos[512:544])

layer2_weights = np.array(pos[544:1568]).reshape(32,32)
layer2_biases = np.array(pos[1568:1600])

layer3_weights = np.array(pos[1600:1632]).reshape(32,1)
layer3_biases = np.array(pos[1632:1633])

network.layers[0].set_weights([layer1_weights, layer1_biases])
network.layers[1].set_weights([layer2_weights, layer2_biases])
network.layers[2].set_weights([layer3_weights, layer3_biases])

In [29]:
pso_algo_predict = model.predict(x_norm_test)

NameError: name 'model' is not defined

In [ ]:
pso_algo_predict[pso_algo_predict >= 0.5] = 1
pso_algo_predict[pso_algo_predict < 0.5] = 0

In [ ]:
print("--------------------- RESULTS OF PARTICLE SWARM OPTIMIZATION -----------------------")
print("------------------------------------------------------------------------")
print("Accuracy Score: ", accuracy_score(y_test, pso_algo_predict))
print("------------------------------------------------------------------------")
print("Mean Squared Error: ", mean_squared_error(y_test, pso_algo_predict))
print("------------------------------------------------------------------------")
print("Confusion Matrix: \n\n",confusion_matrix(y_test, pso_algo_predict))
print("------------------------------------------------------------------------")
print("Classification Report: \n\n",classification_report(y_test, pso_algo_predict))
print("------------------------------------------------------------------------")